Illustration of levels of neuronal network description and predictions 

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import LFPy
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
from matplotlib.collections import PolyCollection, PatchCollection
from plotting import annotate_subplot
import plotting
import example_network_parameters as params
import scipy.signal as ss

In [ ]:
plt.xkcd(scale=0.5)
# from matplotlib import patheffects
# plt.rcParams.update({
#     'path.effects': [], # patheffects.withStroke(linewidth=4, foreground="w")]
# })

In [ ]:
plt.rcParams.update(plotting.rcParams)
golden_ratio = plotting.golden_ratio
figwidth = plotting.figwidth

In [ ]:
plt.rcParams.update({
    'font.family': plt.rcParamsDefault['font.family'],
})

In [ ]:
# E and I colors
colors = ['tab:blue', 'tab:red']

In [ ]:
# Dataset
OUTPUTPATH = os.path.join('output', 'adb947bfb931a5a8d09ad078a6d256b0')
OUTPUTPATH_APPROX = os.path.join('output', 'f0a4032067f1fef454d5369276fb1cc9')

In [ ]:
SEED = 1234

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(figwidth, figwidth))
fig = plt.figure(figsize=(figwidth, figwidth*1.2))
gs = GridSpec(9, 1, hspace=0.4)
ax = fig.add_subplot(gs[:6, 0])
ax.set_title('Signal predictions across scales:')

ax.axis('off')

# ax.axis('equal')
ax.axis((-500, 1500, -1000, 700))


# LFP data
f = h5py.File(os.path.join(OUTPUTPATH, 'RecExtElectrode.h5'), 'r')
data = f['data']['imem'][::2, -1601:]
data = (data.T - data.mean(axis=-1)).T
t = np.linspace(200, 500, data.shape[1])

# approximated data
f = h5py.File(os.path.join(OUTPUTPATH_APPROX, 'RecExtElectrode.h5'), 'r')
data_approx = f['data']['imem'][::2, -1601:]
data_approx = (data_approx.T - data_approx.mean(axis=-1)).T


###############################
# Draw MC+VC network
###############################
ax.text(-350, 0, 'multicompartment\nneuron networks', rotation='90', ha='center', va='center', fontweight='bold')
np.random.seed(SEED)
N_Y = [16, 8]
labels = ['E', 'I']
morphologies = ['BallAndSticks_short.hoc', 'BallAndSticks_I.hoc']
coords = []
for i, (N, label, morphology) in enumerate(zip(N_Y, labels, morphologies)):
    # soma locations
    cell_xz = np.c_[(np.random.rand(N) - 0.5) * 500, (np.random.rand(N) - 0.5) * 200]
    coords.append(cell_xz)

    for j, (x, z) in enumerate(cell_xz):
        morphology=morphology
        cellParameters = dict(
            morphology=morphology,
            templatefile='BallAndSticksTemplate.hoc',
            templatename='BallAndSticksTemplate',
            templateargs=None,
            delete_sections=True,
        )
        cell = LFPy.TemplateCell(**cellParameters)
        cell.set_pos(x, 0, z)

        # show morphology
        zips = []
        for x, z in cell.get_idx_polygons(projection=('x', 'z')):
            zips.append(list(zip(x, z)))
        polycol = PolyCollection(zips,
                                 edgecolors='k',
                                 facecolors=colors[i],
                                 linewidths=1,
                                 zorder=np.random.rand()-0.5,
                                 label=label if j == 0 else '__nolabel__')
        ax.add_collection(polycol)

# illustrate some connections
for h, pre_xz in enumerate(coords):
    for k, post_xz in enumerate(coords):
        for i, (x_post, z_post) in enumerate(post_xz):
            z_syn = [z_post] * len(post_xz)
            for j, (x_pre, z_pre) in enumerate(pre_xz):
                if i != j:
                    if np.random.rand() > 0.9:
                        # draw line from presyn soma to synapse
                        ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*10)], 
                                [z_pre,  z_syn[i] + np.random.randn()*50], 'o-', color=colors[h], 
                               path_effects=[],
                               zorder=np.random.randn()-0.5,
                               lw=1)
                    continue

# legend
ax.legend(loc='center', bbox_to_anchor=(0.4, 0.7))

# probe
el = [list(zip([-40, -40, 0, 40, 40, -40], 
               [500, -50, -100, -50, 600, 600]))]
elpoly = PolyCollection(el,
                        edgecolors='k',
                        facecolors='gray', zorder=0)
ax.add_collection(elpoly)
ax.plot(np.zeros(data.shape[0]), np.linspace(-50, 450, data.shape[0]), 'o', color='k', zorder=0)

# connections
style = "Simple, tail_width=3, head_width=10, head_length=6"
# E outgoing
kw = dict(arrowstyle=style, edgecolor="k", facecolor='w')
arrow = mpatches.FancyArrowPatch((-0, 600), (380, 650),
                             connectionstyle="arc3,rad=-0.2", **kw)
ax.add_patch(arrow)



# draw extracellular traces
for i, x in enumerate(data):
    ax.plot(t + 200, x*1e2 - i * 50 + 600, 'k', lw=1)
ax.text(550, 650, r'brain signals', ha='center', va='center')


# draw approximated extracellular traces
for i, x in enumerate(data_approx):
    ax.plot(t + 950, x*1e2 - i * 50 - 450, 'k', lw=1)
ax.text(1300, -400, 'approximated\nbrain signals', ha='center', va='center')



###############################
# Draw point-neuron network
###############################
ax.text(-350, -500, 'point-neuron\nnetworks', rotation='vertical', ha='center', va='center', fontweight='bold')
np.random.seed(SEED)
N_Y = [16, 8]
morphologies = ['BallAndSticks_short.hoc', 'BallAndSticks_I.hoc']
coords = []
for i, (N, morphology, marker) in enumerate(zip(N_Y, morphologies, ['^', 'o'])):
    # soma locations
    cell_xz = np.c_[(np.random.rand(N) - 0.5) * 500, (np.random.rand(N) - 0.5) * 200 - 500]
    coords.append(cell_xz)
    
    for x, z in cell_xz:
        zorder = np.random.rand() - 0.5
        ax.plot(x, z, marker=marker, mec='k', mfc=colors[i], ms=15, zorder=zorder)
    

# illustrate some connections
for h, pre_xz in enumerate(coords):
    for k, post_xz in enumerate(coords):
        for i, (x_post, z_post) in enumerate(post_xz):
            z_syn = [z_post] * len(post_xz)
            for j, (x_pre, z_pre) in enumerate(pre_xz):
                if i != j:
                    if np.random.rand() > 0.9:
                        # draw line from presyn soma to synapse
                        ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                                [z_pre,  z_syn[i]], 'o-', color=colors[h], 
                               path_effects=[],
                               lw=1)
                    continue
        
        
        
#########################
# Neural mass model illustration
#########################
z = -850
ax.text(-350, z, 'population\nmodels', rotation='vertical', ha='center', va='center', fontweight='bold')
for i, (x, nverts, radius, label) in enumerate(zip([-150, 150],
                               [3, 100],
                               [100, 75],
                               labels)):
    polygon = mpatches.RegularPolygon((x, z), nverts, radius, orientation=0, 
                                      edgecolor=colors[i], facecolor='w', lw=3)
    ax.add_patch(polygon)
    ax.text(x, z, label, color=colors[i], size=32, ha='center', va='center')

    
# connections
style = "Simple, tail_width=3, head_width=10, head_length=6"
# E outgoing
kw_E = dict(arrowstyle=style, color=colors[0])
arrow = mpatches.FancyArrowPatch((-120, -800), (80, -800),
                             connectionstyle="arc3,rad=-0.5", **kw_E)
ax.add_patch(arrow)
arrow = mpatches.FancyArrowPatch((-180, -800), (-230, -880),
                             connectionstyle="arc3,rad=2.0", **kw_E)
ax.add_patch(arrow)

# I outgoing
kw_I = dict(arrowstyle=style, color=colors[1])
arrow = mpatches.FancyArrowPatch((100, -900), (-50, -900),
                             connectionstyle="arc3,rad=-0.5", **kw_I)
ax.add_patch(arrow)
arrow = mpatches.FancyArrowPatch((205, -900), (215, -800),
                             connectionstyle="arc3,rad=2", **kw_I)
ax.add_patch(arrow)


############
# Biophysical detail arrow
############
arrow = mpatches.FancyArrowPatch((-425, -800), (-425, 500),
                             connectionstyle="arc3,rad=-0.", **kw)
ax.add_patch(arrow)
ax.text(-435, -150, 'biophysical detail', rotation='vertical', ha='right', va='center')


#################
# Bridging scales
##################
# MC<->Point neurons
arrow = mpatches.FancyArrowPatch((5, -375), (5, -250),
                             connectionstyle="arc3,rad=0.2", ls=':', **kw)
ax.add_patch(arrow)
arrow = mpatches.FancyArrowPatch((-5, -250), (-5, -375),
                             connectionstyle="arc3,rad=0.2", **kw)
ax.add_patch(arrow)
ax.text(30, -312.5, 'parameter\nmapping', ha='left', va='center')

# Point-neurons <-> Mass models
arrow = mpatches.FancyArrowPatch((5, -725), (5, -600),
                             connectionstyle="arc3,rad=0.2", ls=':', **kw)
ax.add_patch(arrow)
arrow = mpatches.FancyArrowPatch((-5, -600), (-5, -725),
                             connectionstyle="arc3,rad=0.2", **kw)
ax.add_patch(arrow)
ax.text(30, -662.5, 'parameter\nmapping', ha='left', va='center')



############################
# Predictions
############################
# MC + VC networks
x = 750
signals = [
    'extracellular potentials',
    'ECoG',
    'current dipole moment',
    'EEG',
    'MEG'
]
signals.reverse()
# ax.text(x-50, len(signals) * 50 + 200, 'multicompartment\nneuron networks:', weight='bold')
for i, signal in enumerate(signals):
    ax.text(x, i * 50 + 350, '- ' + signal, color='C2')

'''
signals = [
    'spikes',
    'membrane potentials',
    'axial currents',
    'transmembrane currents',
]
for i, signal in enumerate(signals):
    ax.text(x, i * 50 + 0, '- ' + signal, color='k')

    
# Point neuron networks
signals = [
    'spikes',
    "'soma' potentials",
]
ax.text(x-50, len(signals) * 50 - 400, 'point-neuron\nnetworks:', weight='bold')
for i, signal in enumerate(signals):
    ax.text(x, i * 50 - 400, '- ' + signal)

# Neural mass models
signals = [
    'averaged spike rates',
    "averaged 'soma' potentials",
]
ax.text(x-50, len(signals) * 50 - 950, 'population\nmodels:', weight='bold')
for i, signal in enumerate(signals):
    ax.text(x, i * 50 - 950, '- ' + signal)
'''

########## 
# Signals
##########

# Soma potentials
for j, (label, N, z_) in enumerate(zip(labels, [8, 2], [150, -50])):
    with h5py.File(os.path.join(OUTPUTPATH, 'somav.h5'), 'r') as f:
        V = f[label][:N, -1601:]
    
    for i, x in enumerate(V):
        ax.plot(t + 200, x - i * 25 + z_, colors[j], lw=1)
ax.text(550, 150, 'soma potentials', ha='center', va='center')
        
        
# spike trains
T = [params.networkParameters['tstop'] - 100, params.networkParameters['tstop']]
with h5py.File(os.path.join(OUTPUTPATH, 'spikes.h5'), 'r') as f:
    for i, (y, N_y, sgn) in enumerate(zip(labels, [900, 128], [1, -1])):
        times = []
        gids = []

        for g, t_ in zip(f[y]['gids'][()][:N_y], f[y]['times'][:N_y]):
            times = np.r_[times, t_]
            gids = np.r_[gids, np.zeros(t_.size) + g]

        gids = gids[times >= T[0]]
        times = times[times >= T[0]]

        ii = (times >= T[0]) & (times <= T[1])
        t_ = times[ii]
        t_ -= T[0]
        t_ *= 3
        ax.plot(t_ + 400, np.argsort(gids[ii])*sgn - 500, '|', 
                color=colors[i],
                ms=5,
                )
ax.text(550, -250, r'spike trains', ha='center', va='center')

# population averaged Vsoma
for j, label in enumerate(labels):
    with h5py.File(os.path.join(OUTPUTPATH, 'somav.h5'), 'r') as f:
        V = np.mean(f[label][:, -101:], axis=0)
    V = (V.T - V.mean(axis=0)).T
    ax.plot(t[::16] + 200, V*20 - j * 50 - 900, colors[j], lw=1)
ax.text(550, -850, "'soma' potentials", ha='center', va='center')

    
# spike rates
w = ss.windows.gaussian(161, 16)
w /= w.sum()
Delta_t = 2**-4
bins = np.linspace(T[0], T[1], int(np.diff(T) / Delta_t + 1))
with h5py.File(os.path.join(OUTPUTPATH, 'spikes.h5'), 'r') as f:
    for j, Y in enumerate(labels):
        times = []

        for t_ in f[Y]['times']:
            times = np.r_[times, t_]
        
        ii = (times >= T[0]) & (times <= T[1])
        hist, _ = np.histogram(times[ii], bins=bins)
        
        # smoothen rate profiles
        hist = np.convolve(hist, w, 'same')
        hist = hist - hist.mean()
        ax.plot(t[:-1] + 200, hist*50 - j * 50 - 700, color=colors[j], lw=1)
ax.text(550, -650, 'firing rates', ha='center', va='center')


# BLACK BOX (LFPykernels)
blackbox = mpatches.Rectangle((840, -710), 170, 170, edgecolor='k', facecolor='k', lw=3)
ax.text(925, -625, 'Black\nBox', fontsize=20, va='center', ha='center')
ax.add_patch(blackbox)

        
        
# MOAR arrows
# MC -> V_soma
arrow = mpatches.FancyArrowPatch((275, 0), (375, 0), connectionstyle="arc3,rad=0.", ls=':', **kw)
ax.add_patch(arrow)
# Point -> V_soma
arrow = mpatches.FancyArrowPatch((275, -500), (375, -50), connectionstyle="arc3,rad=0.", ls=':', **kw)
ax.add_patch(arrow)
# MC -> spikes
arrow = mpatches.FancyArrowPatch((275, 0), (375, -450), connectionstyle="arc3,rad=0.", **kw)
ax.add_patch(arrow)
# Point -> spikes
arrow = mpatches.FancyArrowPatch((275, -500), (375, -500), connectionstyle="arc3,rad=0.", **kw)
ax.add_patch(arrow)
# spikes -> rates
arrow = mpatches.FancyArrowPatch((375, -600), (375, -725), connectionstyle="arc3,rad=1", **kw)
ax.add_patch(arrow)
# Pop -> rates
arrow = mpatches.FancyArrowPatch((325, -850), (375, -775), connectionstyle="arc3,rad=0.", **kw)
ax.add_patch(arrow)
# Pop -> V_soma
arrow = mpatches.FancyArrowPatch((325, -850), (375, -925), connectionstyle="arc3,rad=0.", ls=':', **kw)
ax.add_patch(arrow)

# spikes->blackbox
arrow = mpatches.FancyArrowPatch((725, -500), (825, -615), connectionstyle="arc3,rad=0.2", **kw)
ax.add_patch(arrow)

# rates->blackbox
arrow = mpatches.FancyArrowPatch((725, -700), (825, -635), connectionstyle="arc3,rad=-0.2", **kw)
ax.add_patch(arrow)

# blackbox->approximations
arrow = mpatches.FancyArrowPatch((1025, -625), (1125, -625), connectionstyle="arc3,rad=0.", **kw)
ax.add_patch(arrow)

ax.text(-450, 700, 'A', 
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=20,
        fontweight='bold')




################## 

# fig, ax = plt.subplots(1, 1, figsize=(figwidth, figwidth / 3))
ax = fig.add_subplot(gs[6:, 0])
ax.set_title("Unraveling the Black Box:")

ax.axis('off')

# ax.axis('equal')
ax.axis((-1000, 3000, -4500, -2000))

xoffsets = [500, -500, -1500]
yoffsets = [2800] * 3
titles = ['multicompartment\nneuron networks', 'hybrid scheme', 'kernel predictor']
arrowlabels = ['reduction', 'reduction', 'application']

bulletlists = ['\n'.join(['input:',
                          '- network specifications',
                          '- neuron specifications',
                          'output:',
                          '- brain signals', 
                          '- spike trains', 
                          '- inferred spike rates',
                          '- membrane potentials']),
               '\n'.join(['input:', 
                          '- network specifications',
                          '- neuron specifications', 
                          '- linearized conductances',
                          '- spike trains',
                          'output:',
                          '- approx. signals', 
                          '- filter kernels',
                          r'$\rightarrow$ linear model validation']),
               '\n'.join(['input:',
                          '- network specifications',
                          '- neuron specifications', 
                          '- linearized conductances',
                          '- 1 neuron per pathway',
                          'output:',
                          '- filter kernels'])]

###############################
# Draw MC+VC networks
###############################
for ii, (title, arrowlabel, bullets, xoffset, yoffset) in enumerate(zip(titles, arrowlabels, bulletlists, xoffsets, yoffsets)):
    ax.text(-250-xoffset, 650 - yoffset, title, fontweight='normal', ha='left')
    np.random.seed(SEED)
    if ii == 2:
        N_Y = [1, 1]
    else:
        N_Y = [16, 8]
    labels = ['E', 'I']
    morphologies = ['BallAndSticks_short.hoc', 'BallAndSticks_I.hoc']
    coords = []
    for i, (N, label, morphology) in enumerate(zip(N_Y, labels, morphologies)):
        # soma locations
        if ii == 2:
            cell_xz = np.array([[i * 250 - 125 - xoffset], [-yoffset]]).T
        else:
            cell_xz = np.c_[(np.random.rand(N) - 0.5) * 500 - xoffset, (np.random.rand(N) - 0.5) * 200 - yoffset]
        coords.append(cell_xz)

        for j, (x, z) in enumerate(cell_xz):
            morphology=morphology
            cellParameters = dict(
                morphology=morphology,
                templatefile='BallAndSticksTemplate.hoc',
                templatename='BallAndSticksTemplate',
                templateargs=None,
                delete_sections=True,
            )
            cell = LFPy.TemplateCell(**cellParameters)
            cell.set_pos(x, 0, z)

            # show morphology
            zips = []
            for x, z in cell.get_idx_polygons(projection=('x', 'z')):
                zips.append(list(zip(x, z)))
            polycol = PolyCollection(zips,
                                    edgecolors='k',
                                    facecolors=colors[i],
                                    linewidths=1,
                                    zorder=np.random.rand()-0.5,
                                    label=label if j == 0 else '__nolabel__')
            ax.add_collection(polycol)

    # illustrate some connections
    if ii == 0:
        for h, pre_xz in enumerate(coords):
            for k, post_xz in enumerate(coords):
                for i, (x_post, z_post) in enumerate(post_xz):
                    z_syn = [z_post] * len(post_xz)
                    for j, (x_pre, z_pre) in enumerate(pre_xz):
                        if i != j:
                            if np.random.rand() > 0.9:
                                # draw line from presyn soma to synapse
                                ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*10)], 
                                        [z_pre,  z_syn[i] + np.random.randn()*50], 'o-', color=colors[h], 
                                    path_effects=[],
                                    zorder=np.random.randn()-0.5,
                                    lw=0.5, ms=3)
                            continue

    # probe
    el = np.array([list(zip([-40, -40, 0, 40, 40, -40], 
                [500, -50, -100, -50, 600, 600]))])
    el[0, :, 0] -= xoffset
    el[0, :, 1] -= yoffset
    elpoly = PolyCollection(el,
                            edgecolors='k',
                            facecolors='gray', zorder=0)
    ax.add_collection(elpoly)
    ax.plot(np.zeros(data.shape[0]) - xoffset, np.linspace(-50, 450, data.shape[0]) - yoffset, 'o', color='k', zorder=0, ms=3)


    # reductionist arrows and annotations
    arrow = mpatches.FancyArrowPatch((250 - xoffset, -yoffset), (750 - xoffset, -yoffset), connectionstyle="arc3,rad=0.", **kw)
    ax.add_patch(arrow)
    ax.text(500 - xoffset, 50 - yoffset, arrowlabel, ha='center')

    # bullets
    # arrow = mpatches.FancyArrowPatch((-100 - xoffset, -200-yoffset), (-250 - xoffset, -400-yoffset), connectionstyle="arc3,rad=0.5", **kw)
    # ax.add_patch(arrow)
    ax.text(-250 - xoffset, -400 - yoffset, bullets, ha='left', va='top')


# bypass hybrid scheme arrow
arrow = mpatches.FancyArrowPatch((250 - 500, -yoffset - 50), (1750 - 500, -yoffset - 50), connectionstyle="arc3,rad=0.2", **kw)
ax.add_patch(arrow)


# panel E stuff
xoffset = -2500

# title
ax.text(2250, 650 - yoffset, 'kernel-based signal\npredictions', fontweight='normal', ha='left')

# bullets
bullets = '\n'.join([
    'input:',
    '- spike rates',
    '- kernels',
    'output:',
    '- approx. signals',
    r'$\rightarrow$ validation'
])
ax.text(-250 - xoffset, -400 - yoffset, bullets, ha='left', va='top')

# ax.axis('tight')

ax.text(
    -900, -2000, 'B', 
    horizontalalignment='center',
    verticalalignment='center',
    fontsize=20,
    fontweight='bold')




if not os.path.isdir('figures'):
    os.mkdir('figures')
fig.savefig(os.path.join('figures', 'figure01.pdf'), bbox_inches='tight')

In [ ]:
plt.rcParams